# Python TD 4

## Python et le web

<br></br>
## Exercice 1

Écrire un script qui prend comme paramètre une url et télécharge dans le répertoire courant toutes les images contenues dans la page.

On utilisera urllib.request pour récupérer le code html, une classe dérivée de html.parser pour récupérer les attributs src de toutes les balises img, et le module urllib.parse pour construire l'url de téléchargement de chaque image.

Comme des images provenant de sites différents peuvent avoir le même nom, on utilisera le module time pour préfixer au nom du fichier la date et l'heure du téléchargement. Le résultat pourra ressembler à ceci :

```bash
$ dlimgs http://www-igm.univ-mlv.fr/~jyt/
$ ls -l
-rw-r--r-- 1 jyt adm    130 janv.  5 16:10 2014-01-05_16-10-14_barre.gif
-rw-r--r-- 1 jyt adm   1587 janv.  5 16:10 2014-01-05_16-10-14_iuf.gif
-rw-r--r-- 1 jyt adm   2344 janv.  5 16:10 2014-01-05_16-10-14_logoCNRS.jpg
-rw-r--r-- 1 jyt adm   4153 janv.  5 16:10 2014-01-05_16-10-14_logoigm.gif
-rw-r--r-- 1 jyt adm    553 janv.  5 16:10 2014-01-05_16-10-14_mail.gif
-rw-r--r-- 1 jyt adm  13462 janv.  5 16:10 2014-01-05_16-10-14_thibon.gif
-rw-r--r-- 1 jyt adm   4554 janv.  5 16:10 2014-01-05_16-10-14_titre-petit.jpg
-rw-r--r-- 1 jyt adm   3196 janv.  5 16:10 2014-01-05_16-10-15_scholar_sm.gif
```
On pourra ensuite ajouter des options ad libitum, par exemple un répertoire de téléchargement, un préfixe autre que la date et l'heure, etc.

In [70]:
import os
import time
from html.parser import HTMLParser
from urllib.request import urlopen, urlretrieve
from urllib.parse import urljoin

class URLLister(HTMLParser):
    def reset(self):
        HTMLParser.reset(self)
        self.urls = []

    def handle_starttag(self, tag, attrs):
        src = [v for k, v in attrs if k == 'src']
        if src:
            self.urls.extend(src)

def download_images(url):
    try:
        f = urlopen(url).read().decode('utf8')
    except Exception as e:
        print(f"Erreur lors de l'ouverture de l'URL : {e}")
        return

    parser = URLLister()
    parser.feed(f)
    parser.close()

    for img_url in parser.urls:
        
        full_url = urljoin(url, img_url)

        img_name = os.path.basename(full_url)
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        download_name = f"{timestamp}_{img_name}"

        try:
            urlretrieve(full_url, download_name)
            print(f"Téléchargé : {download_name}")
        except Exception as e:
            print(f"Erreur lors du téléchargement de {full_url} : {e}")

# Exemple d'utilisation
url = 'http://www-igm.univ-mlv.fr/~jyt/'
download_images(url)


Téléchargé : 2024-10-25_15-31-11_logoigm.gif
Téléchargé : 2024-10-25_15-31-11_titre-petit.jpg
Téléchargé : 2024-10-25_15-31-11_barre.gif
Téléchargé : 2024-10-25_15-31-11_logoCNRS.jpg
Téléchargé : 2024-10-25_15-31-11_thibon.gif
Téléchargé : 2024-10-25_15-31-11_mail.gif
Téléchargé : 2024-10-25_15-31-11_iuf.gif
Téléchargé : 2024-10-25_15-31-11_scholar_sm.gif


<br></br>

## Exercice 2

Lorsque les services web ont été introduits, il existait quelque part en Suède un serveur permettant aux développeurs de tester leurs clients (pour le protocole SOAP). Il renvoyait des injures aléatoires puisées dans le répertoire du capitaine Haddock (en anglais). Cette institution ayant disparu, nous avons installé (provisoirement) un service similaire sur le serveur de l'IGM, http://monge.univ-mlv.fr:8888.

Pour n'utiliser que les ressources natives de Python, nous avons employé le protocole xmlrpc (qui est l'ancêtre de SOAP) de sorte que l'accès au serveur ne nécéssitera que deux ou trois lignes de Python, l'une d'entre elles consistant à importer une classe du module xmlrp.client.

0) Parcourir rapidement la documentation du module xmlrpc.client.

1) Le serveur accepte l'introspection. Découvrir ses méthodes, et les essayer.

2) En utilisant le module xmlrpc.server, écrire un serveur identique et le tester sur "localhost".

La documentation est ici pour le client, et là pour le serveur.

In [83]:
import xmlrpc.client

proxy = xmlrpc.client.ServerProxy("http://monge.univ-mlv.fr:8888/")
methods = proxy.system.listMethods()
dir(proxy)
print(proxy.jure())
print(methods)

ROltb24K
['__', 'help', 'jure', 'system.listMethods', 'system.methodHelp', 'system.methodSignature', 'tous_les_jurons']


In [ ]:
from xmlrpc.server import SimpleXMLRPCServer


with SimpleXMLRPCServer(('localhost', 8001)) as server:

    # Register a function under a different name
    def jure():
        return "Ugly"
    server.register_instance(jure())

    print("Server is running on port 8001...")

    # Run the server's main loop
    server.serve_forever()

Server is running on port 8001...


In [ ]:
import xmlrpc.client

server = xmlrpc.client.ServerProxy('http://localhost:8001/')

result = server.jure()
print(result)
